<a href="https://colab.research.google.com/github/fatemafaria142/Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/Mental_Health_Advice_Generation_using_Starling_LM_7B_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [2]:
from datasets import load_dataset
instruct_tune_dataset = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**
* The dataset contains two columns.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Question:", data['Context'])
    print("Response:", data['Response'])
    print("\n-----------------------------\n")


Data Point 1:
Question: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Response: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [6]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 300
    })
})

In [35]:
def create_prompt(sample):
    # Use a predefined template for instructions
    prompt = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: "
    prompt += sample['Context']
    single_turn_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant: {sample['Response']}"
    return single_turn_prompt

In [36]:
create_prompt(instruct_tune_dataset["train"][0])

"GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the sam

In [37]:
create_prompt(instruct_tune_dataset["train"][1])

'GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you\'re all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here.\n   I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant: Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone.\xa0Changing our feelings is like changing our thought

In [38]:
create_prompt(instruct_tune_dataset["train"][2])

"GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant: First thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with 

In [39]:
create_prompt(instruct_tune_dataset["train"][3])

"GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant: Therapy is essential for those that are feeling depressed and worthless. When I work with those that are experiencing concerns related to feeling of depression and issues with self esteem. I generally work with my client to help build coping skills to reduce level of depression and to ass

In [40]:
create_prompt(instruct_tune_dataset["train"][4])

"GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant: I first want to let you know that you are not alone in your feelings and there is always someone there to help. You can always change your feelings and change your way of thinking by being open to trying to change. You can always make yourself available to learning new things or volunteer

In [41]:
create_prompt(instruct_tune_dataset["train"][5])

"GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant: Heck, sure thing, hun!Feelings of 'depression' have a deeply-rooted base in physical structures that may not be functioning very well at present; and, we can certainly turn them on again using means that you are able to find around the house and with relative ease :)After that, emotional 

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [42]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [43]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [16]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [44]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [45]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [46]:
prompt = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|>GPT4 Correct Assistant:"""

In [47]:
generate_response(prompt, model)

'<s> GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you\'re all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here.\n   I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end_of_turn|> GPT4 Correct Assistant: As a mental health counselor, it\'s important to acknowledge and validate your feelings of worthlessness. It\'s not easy to reach out for help and express these thoughts, but I\'m glad you did. It\'s the first step toward understanding and overcoming these struggles.\n\nHere ar

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [48]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [49]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [50]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [51]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [52]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.587500
20,2.026500
30,1.818200
40,1.921700
50,1.842800
60,2.007500
70,1.952600
80,1.886300
90,1.995600
100,2.130300


TrainOutput(global_step=100, training_loss=2.0169007110595705, metrics={'train_runtime': 887.8584, 'train_samples_per_second': 0.451, 'train_steps_per_second': 0.113, 'total_flos': 8741776785408000.0, 'train_loss': 2.0169007110595705, 'epoch': 0.15})

### **Save the model**

In [53]:
trainer.save_model("Starling_instruct_generation")

In [54]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [55]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [68]:
prompt = "GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: "
prompt += "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone? <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone? <|end_of_turn|>  GPT4 Correct Assistant: It's great that you're seeking help and recognizing that these thoughts are affecting your life. Here are a few suggestions from a mental health counselor that might be helpful:

1. Get Support: Reach out to your friends and family so that they can provide you with emotional support and enc

### **Example No:2**

In [70]:
prompt = "GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: "
prompt += "I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|>  GPT4 Correct Assistant: It's really challenging to hear about your struggles with anxiety and depression. You've been willing to reach out for support, and I commend you for taking that brave step. It's never easy to admit that we're not okay or that we need help.

First of all, it's important to remember that this is a team effort. You, me, and your doctor or therapist are in this together to help you lead a better and more fulfilled life. Sometimes it takes a little while to find the right fit with the right mental health profe

### **Example No:3**

In [71]:
prompt = "GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: "
prompt += "I get so much anxiety, and I don’t know why. I feel like I can’t do anything by myself because I’m scared of the outcomes. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I get so much anxiety, and I don’t know why. I feel like I can’t do anything by myself because I’m scared of the outcomes. <|end_of_turn|>  GPT4 Correct Assistant: I'm sorry to hear that you're struggling with anxiety. It can be really challenging to live with, but you've taken the first step in becoming aware and seeking help. Here are some steps you can take to manage your anxiety:

1. Educate Yourself: Understand what anxiety is and what it does to your body. Start to understand the signs and symptoms of anxiety and what triggers it. Read about anxiety, talk to your doctor or another person who has experienced anxiety, or research different typ